# Data Exploration: LastFM Dataset

This notebook provides a simple exploration of the LastFM dataset to understand:
- Dataset structure and schema
- Data quality (missing values, duplicates)
- Basic statistics
- Temporal patterns
- User and artist distributions

In [ ]:
import sys
sys.path.append('..')

from pyspark.sql import functions as F
from src.common.definition import (
    create_spark_session, 
    load_track_data, 
    add_sessions_id_columns
)
from src.analysis_tracks_by_sessions import compute_session_duration
from src.visualization import plot_time_series
import matplotlib.pyplot as plt

DATA_PATH = "../userid-timestamp-artid-artname-traid-traname.tsv"
SESSION_GAP_SEC = 20 * 60

In [4]:
spark = create_spark_session("data_exploration")

your 131072x1 screen size is bogus. expect trouble
JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
track_list = load_track_data(spark, DATA_PATH)

In [ ]:
print("Dataset Schema:")
track_list.printSchema()

In [ ]:
print("Sample Records:")
track_list.show(10, truncate=False)

In [ ]:
total_records = track_list.count()
print(f"Total listening records: {total_records:,}")

In [ ]:
null_counts = track_list.select(
    *[F.count(F.when(F.col(c).isNull(), 1)).alias(c) for c in track_list.columns]
).collect()[0]

print("Null Value Analysis:")
print(f"{'Column':<30} {'Null Count':<15} {'Percentage':<10}")
for col, count_nulls in zip(track_list.columns, null_counts):
    percentage = (count_nulls / total_records) * 100
    print(f"{col:<30} {count_nulls:<15,} {percentage:>8.2f}%")

In [ ]:
print("\nBasic Statistics:")
track_list.describe().show()

In [ ]:
date_range = track_list.select(
    F.min("timestamp").alias("min_date"),
    F.max("timestamp").alias("max_date")
).collect()[0]

print(f"Date Range:")
print(f"  Start: {date_range['min_date']}")
print(f"  End: {date_range['max_date']}")
print(f"  Duration: {(date_range['max_date'] - date_range['min_date']).days} days")

In [ ]:
yearly_activity = (
    track_list
    .withColumn("year", F.year("timestamp"))
    .groupBy("year")
    .agg(F.count("*").alias("track_count"))
    .orderBy("year")
)

print("\nListening Activity by Year:")
yearly_activity.show()

In [ ]:
yearly_df = yearly_activity.toPandas()
plt.figure(figsize=(12, 6))
plt.bar(yearly_df['year'], yearly_df['track_count'])
plt.xlabel('Year')
plt.ylabel('Number of Tracks Played')
plt.title('Listening Activity by Year')
plt.xticks(yearly_df['year'])
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
n_users = track_list.select("userid").distinct().count()
print(f"Total unique users: {n_users:,}")

In [ ]:
user_activity = (
    track_list
    .groupBy("userid")
    .agg(F.count("*").alias("track_count"))
    .select("track_count")
)

print("\nUser Activity Statistics:")
user_activity.describe().show()

In [ ]:
top_artists = (
    track_list
    .groupBy("artist_name")
    .agg(F.count("*").alias("play_count"))
    .orderBy(F.desc("play_count"))
    .limit(20)
)

print("\nTop 20 Most Popular Artists:")
top_artists.show(truncate=False)

In [ ]:
top_tracks = (
    track_list
    .groupBy("track_name", "artist_name")
    .agg(F.count("*").alias("play_count"))
    .orderBy(F.desc("play_count"))
    .limit(20)
)

print("\nTop 20 Most Played Tracks:")
top_tracks.show(truncate=False)

In [ ]:
track_list_sessions = add_sessions_id_columns(track_list, SESSION_GAP_SEC)
n_sessions = track_list_sessions.select("userid", "session_id").distinct().count()

print(f"Total sessions (20-minute gap): {n_sessions:,}")
print(f"Average tracks per session: {total_records / n_sessions:.2f}")

In [ ]:
sessions_duration = compute_session_duration(track_list_sessions)

print("\nSession Duration Statistics:")
sessions_duration.select("session_duration_sec").describe().show()

print("\nSession Duration in Hours:")
sessions_duration.select(
    (F.col("session_duration_sec") / 3600).alias("session_duration_hours")
).describe().show()

In [ ]:
print("\nTop 10 Longest Sessions:")
sessions_duration.orderBy(F.desc("session_duration_sec")).limit(10).show()

In [ ]:
monthly_activity = (
    track_list
    .withColumn("month", F.date_trunc("month", "timestamp"))
    .groupBy("month")
    .agg(F.count("*").alias("track_count"))
    .orderBy("month")
)

monthly_df = monthly_activity.toPandas()
monthly_df.set_index('month', inplace=True)

# Plot
plot_time_series(
    monthly_df,
    "track_count",
    title="Monthly Listening Activity",
    ylabel="Number of Tracks Played"
)

In [ ]:
spark.stop()